In [ ]:
import requests
import numpy as np
!pip install "tensorflow>=1.14.0,<2.0"
!pip install "tensorflow-gpu>=1.14.0,<2.0"
!pip install  ampligraph

In [1]:
import ampligraph
ampligraph.__version__
import pandas as pd
import numpy as np

from ampligraph.datasets import load_from_ntriples
from ampligraph.datasets import load_from_csv

In [3]:
Y=load_from_csv('/content','3000data.csv',sep=',',add_reciprocal_rels=False)

In [5]:
from ampligraph.evaluation import train_test_split_no_unseen 

Y_train, Y_test = train_test_split_no_unseen(Y, test_size=150) 

In [6]:
print('Train set size: ', Y_train.shape)
print('Test set size: ', Y_test.shape)

Train set size:  (2664, 3)
Test set size:  (150, 3)


In [38]:
from ampligraph.latent_features import ConvE

In [39]:
model = ConvE(batches_count=100, 
                seed=0, 
                epochs=20, 
                k=10, 
                eta=2,
              embedding_model_params={'conv_filters': 32, 'conv_kernel_size': 3, 'dropout_conv': 0.3, 'dropout_dense': 0.2, 'dropout_embed': 0.2, 'use_batchnorm': True, 'use_bias': True}
                ,optimizer='adam', 
                optimizer_params={'lr':0.0005},
                loss='bce', 
                regularizer=None, 
                regularizer_params={}, 
                verbose=True)

KeyError: ignored

In [9]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

model.fit(Y_train, early_stopping = False)

Average Loss:  82.066937: 100%|██████████| 20/20 [00:15<00:00,  1.33epoch/s]


In [10]:
from ampligraph.utils import create_tensorboard_visualizations

In [12]:
model.get_embeddings(['The World Health Organization','outbreak'], embedding_type='entity')

array([[-0.40424192,  0.02792663, -0.05198541, -0.00910034,  0.00590181,
        -0.21773404, -0.18012853, -0.00072256, -0.28044775, -0.22150037],
       [-0.456721  , -0.00531049, -0.01225822, -0.00084796, -0.06108423,
        -0.42588934, -0.4307937 ,  0.00052859, -0.4015078 , -0.43181518]],
      dtype=float32)

In [13]:
model.get_embeddings(['common symptoms','cough'], embedding_type='entity')

array([[-0.29904914, -0.02632174, -0.05305538, -0.01018774, -0.01145934,
        -0.34290138, -0.13542995, -0.05750841, -0.43179226, -0.1398689 ],
       [-0.28489643,  0.00321492,  0.00314562, -0.01331351, -0.02827432,
        -0.41970393, -0.13677768,  0.01145979, -0.34569067, -0.2720827 ]],
      dtype=float32)

In [33]:
positives_filter = Y

In [17]:
from ampligraph.discovery import query_topn

In [18]:
query_topn(model, top_n=5,
            head=None, relation='include', tail='cough',
           ents_to_consider=None, rels_to_consider=None)

Mapping set data: test


100%|██████████| 4272/4272 [00:02<00:00, 1426.33it/s]


(array([['People', 'include', 'cough'],
        ['patients', 'include', 'cough'],
        ['cases', 'include', 'cough'],
        ['Bessie Delany', 'include', 'cough'],
        ['number', 'include', 'cough']], dtype='<U153'),
 array([0.02173591, 0.02153608, 0.02152413, 0.02141112, 0.02135545],
       dtype=float32))

In [15]:
from ampligraph.latent_features import save_model, restore_model

In [16]:
save_model(model, './covid_model.pkl')

In [11]:
create_tensorboard_visualizations(model,r"./")

KeyError: ignored

In [19]:
from ampligraph.evaluation import evaluate_performance
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score

In [37]:
positives_filter = Y_train
ranks = evaluate_performance(Y_test,
                             model=model,
                             filter_triples=positives_filter,
                             use_default_protocol=True,
                             verbose=True)

WARNING - DeprecationWarning: use_default_protocol will be removed in future. Please use corrupt_side argument instead.
Mapping set data: test
Mapping set data: filter


100%|██████████| 932/932 [01:49<00:00,  8.50batch/s]


In [22]:
mrr = mrr_score(ranks)
mr = mr_score(ranks)
print("MRR: %.2f" % (mrr))
print("MR: %.2f" % (mr))

hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10: %.2f" % (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3: %.2f" % (hits_3))
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1: %.2f" % (hits_1))

MRR: 0.02
MR: 1491.72
Hits@10: 0.05
Hits@3: 0.02
Hits@1: 0.00


In [34]:
X_unseen = np.array([
    ['guidelines', 'recommend', 'medication'],
  ['black musician', 'wears', 'royal livery'],
   ['Common effects', 'include', 'chest infections'],
])

In [35]:
unseen_filter = np.array(list({tuple(i) for i in np.vstack((positives_filter, X_unseen))}))

In [36]:
ranks_unseen = evaluate_performance(
    X_unseen, 
    model=model, 
    filter_triples=unseen_filter,   # Corruption strategy filter defined above 
    corrupt_side = 's+o',
    use_default_protocol=False, # corrupt subj and obj separately while evaluating
    verbose=True
)

Mapping set data: test
Mapping set data: filter


100%|██████████| 932/932 [01:49<00:00,  8.52batch/s]


UnboundLocalError: ignored

In [26]:
scores = model.predict(X_unseen)

Mapping set data: test


100%|██████████| 3/3 [00:00<00:00, 34.24it/s]


In [27]:
from scipy.special import expit
probs = expit(scores)

In [28]:
pd.DataFrame(list(zip([' '.join(x) for x in X_unseen], 
                      ranks_unseen, 
                      np.squeeze(scores),
                      np.squeeze(probs))), 
             columns=['statement', 'rank', 'score', 'prob']).sort_values("score")

NameError: ignored